In [2]:
%pip install --upgrade --quiet boto3 botocore langchain tiktoken unstructured python-dotenv

In [3]:
import boto3
import os
import json
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [4]:
bedrock_runtime = boto3.client(
    service_name = "bedrock-runtime",
    region_name = "us-east-1"
)

In [5]:
#modelId = 'anthropic.claude-instant-v1'
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

In [6]:
from langchain.document_loaders import S3FileLoader

In [7]:
loader = S3FileLoader("cur-data-diptiman", "cur_data.csv")

In [9]:
doc = loader.load()

In [13]:
prompt_cur = f"""
    \n\nHuman: Write a financial summary of the report.
    {doc}
    
    Assistant:
"""

In [14]:
body = json.dumps({
                    "prompt": prompt_cur,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [15]:
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a summary of the key financial information from the report:

- Total AWS costs: $1,347.4436 

- Largest cost item: 
    - Amazon EC2 m4.10xlarge instance usage: $1060.1256

- Other major cost items:
    - Amazon EC2 m3.xlarge on-demand instance usage: $5.3200
    - Amazon EC2 m3.xlarge reserved instance usage: $0.3000 

- Costs broken down by application:
    - App A: $5.3200
    - App B: $0.3000

- Time period covered:
    - August 1, 2019 00:00:00 to August 31, 2019 23:59:59

The report covers Amazon EC2 instance usage across m3.xlarge and m4.10xlarge instance types, both on-demand and reserved. Total costs are $1,347.44 over the month of August 2019. The largest cost item is m4.10xlarge on-demand instance usage at $1060.13. Costs are broken down by two applications, with App A accounting for $5.32 and App B accounting for $0.30.
